In [1]:
import pandas as pd
import os
import shapely.geometry as sg
data_directory = os.path.join('..', 'data', 'ptv', '20240224')

In [2]:
branch_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '10', '11']
table_names = ['stop_times', 'stops', 'trips', 'routes', 'calendar', 'calendar_dates', 'agency', 'shapes']

In [3]:
def get_df(branch_id, table_name):
    files = [os.path.join(data_directory, f) for f in os.listdir(data_directory) if f.split('-')[1] == str(branch_id) and f.split('-')[2] == table_name]
    if len(files) == 0:
        return None
    return pd.concat([pd.read_csv(f) for f in files])

In [4]:
df = {branch_id: {table_name: get_df(branch_id, table_name) for table_name in table_names} for branch_id in branch_ids}

In [ ]:
bid = '3'

In [5]:
df_trams = df['3']['stop_times'].groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()
df_trams = pd.merge(df['3']['trips'], df_trams, on='trip_id')
df_trams = pd.merge(df['3']['routes'], df_trams, on='route_id')
df_trams['stops_count'] = df_trams['stops_sequence'].apply(lambda x: len(x))
df_trams = df_trams.sort_values(by=['route_short_name', 'stops_count'], ascending=False).drop_duplicates(subset=['route_short_name', 'trip_headsign', 'direction_id'], keep='first')

df_tram_stops = pd.merge(df['3']['stop_times'], df['3']['trips'], on='trip_id')
df_tram_stops = pd.merge(df_tram_stops, df['3']['routes'], on='route_id')
df_tram_stops  = df_tram_stops.groupby(['route_short_name', 'trip_headsign', 'direction_id'])['stop_id'].apply(lambda x: x.unique()).to_frame(name='stops').reset_index()

df_trams = df_trams.merge(df_tram_stops, on=['route_short_name', 'trip_headsign', 'direction_id'])

df_trams['stops_count_2'] = df_trams['stops'].apply(lambda x: len(x))
df_trams['stops_set_1'] = df_trams['stops_sequence'].apply(lambda x: sorted(x))
df_trams['stops_set_2'] = df_trams['stops'].apply(lambda x: sorted(x))

df_trams['stops_set_1_size'] = df_trams['stops_set_1'].apply(lambda x: len(set(x)))
df_trams['stops_set_2_size'] = df_trams['stops_set_2'].apply(lambda x: len(set(x)))

In [6]:
df_trams[df_trams['stops_set_1'] != df_trams['stops_set_2']].iloc[2]['stops_sequence']

array([18038, 18039, 18052, 18040, 18041, 18042, 18043, 18044, 18045,
       18046, 18047, 18048, 18049, 18050, 17880, 20979, 18105, 18174,
       18175, 18176, 18178, 21233, 21232, 18179, 18180, 18181, 18182,
       18183, 18037, 18038], dtype=int64)

In [7]:
df_routes_full = pd.merge(df['3']['trips'], df['3']['routes'], on='route_id', how='left')

In [12]:
df_routes_full = pd.merge(df_routes_full, df['3']['stop_times'], on='trip_id', how='left')

In [8]:
df_trams['anomalies'] = df_trams.apply(lambda x: set(x['stops']) - set(x['stops_sequence']), axis=1)

In [9]:
df_anomalies = df_trams[df_trams['anomalies'].apply(lambda x: len(x) > 0)][['route_short_name', 'trip_headsign', 'direction_id', 'anomalies']]

In [16]:
df_at = df_anomalies.explode('anomalies').merge(df_routes_full, left_on=['route_short_name', 'trip_headsign', 'direction_id', 'anomalies'], right_on=['route_short_name', 'trip_headsign', 'direction_id', 'stop_id']).sort_values(by=['trip_id', 'stop_sequence'], ascending=True)
df_at = df_at.groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()

In [20]:
df_at = df_at.groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()

In [22]:
df_at = pd.merge(df['3']['trips'], df_at, on='trip_id')

In [24]:
df_at['stops_count'] = df_at['stops_sequence'].apply(lambda x: len(x))

In [26]:
df_at = pd.merge(df['3']['routes'], df_at, on='route_id')

In [30]:
df_at = df_at.sort_values(by=['route_short_name', 'stops_count'], ascending=False).drop_duplicates(subset=['route_short_name', 'trip_headsign', 'direction_id'], keep='first').merge(df_anomalies, on=['route_short_name', 'trip_headsign', 'direction_id'], how='left')

In [33]:
df_at[df_at.apply(lambda x: set(x['anomalies']) - set(x['stops_sequence']), axis=1).apply(lambda x: len(x) > 0)]

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color,service_id,trip_id,shape_id,trip_headsign,direction_id,stops_sequence,stops_count,anomalies


In [34]:
df_at = df_at[['route_short_name', 'trip_headsign', 'direction_id', 'trip_id']]

In [36]:
df_at = df_at.merge(df['3']['stop_times'], on='trip_id')

In [38]:
df_at = df_at.groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()

In [40]:
df_at = pd.merge(df['3']['trips'], df_at, on='trip_id')
df_at = pd.merge(df['3']['routes'], df_at, on='route_id')

In [42]:
df_at = pd.concat([df_at, df_trams])

In [44]:
df_at.drop(columns=['anomalies', 'stops_count', 'stops', 'stops_count_2', 'stops_set_1', 'stops_set_2', 'stops_set_1_size', 'stops_set_2_size'], inplace=True)

In [46]:
df_at['stops_count'] = df_at['stops_sequence'].apply(lambda x: len(x))

In [49]:
df_at.sort_values(by=['route_short_name', 'trip_headsign', 'direction_id', 'stops_count'], ascending=False, inplace=True)
df_at.to_csv('trips.csv', index=False)

In [58]:
df['3']['shapes']['point'] = df['3']['shapes'].apply(lambda x: sg.Point(x['shape_pt_lon'], x['shape_pt_lat']), axis=1)

In [59]:
df_lines = df['3']['shapes'].groupby('shape_id')['point'].apply(lambda x: sg.LineString(x.to_numpy())).to_frame(name='line').reset_index()

In [65]:
df['3']['stops']['stop_pt'] = df['3']['stops'].apply(lambda x: sg.Point(x['stop_lon'], x['stop_lat']), axis=1)

In [68]:
df_geo = pd.merge(df['3']['stop_times'], df['3']['trips'][['trip_id', 'shape_id']], on='trip_id')
df_geo = pd.merge(df_geo, df_lines, on='shape_id')
df_geo = df_geo[['shape_id', 'stop_id']].drop_duplicates()
df_geo = pd.merge(df_geo, df_lines, on='shape_id')
df_geo = pd.merge(df_geo, df['3']['stops'][['stop_id', 'stop_pt']], on='stop_id')

In [82]:
df_geo[df_geo.apply(lambda x: x['line'].contains(x['stop_pt']), axis=1)]

,shape_id,stop_id,line,stop_pt
1330,3-109-mjp-5.6.R,17897,LINESTRING (145.122349164379 -37.8178755249435...,POINT (144.974487687436 -37.8123788849696)
1331,3-109-mjp-7.6.R,17897,LINESTRING (145.122349164379 -37.8178755249435...,POINT (144.974487687436 -37.8123788849696)
1332,3-109-mjp-6.6.R,17897,LINESTRING (145.122349164379 -37.8178755249435...,POINT (144.974487687436 -37.8123788849696)
1333,3-109-mjp-3.10.R,17897,LINESTRING (145.122349164379 -37.8178755249435...,POINT (144.974487687436 -37.8123788849696)
1334,3-109-mjp-4.10.R,17897,LINESTRING (145.122349164379 -37.8178755249435...,POINT (144.974487687436 -37.8123788849696)
...,...,...,...,...
20167,3-57-mjp-5.4.R,17948,LINESTRING (144.877988852369 -37.7688291815855...,POINT (144.914787348936 -37.78035451832)
20168,3-57-mjp-7.4.R,17948,LINESTRING (144.877988852369 -37.7688291815855...,POINT (144.914787348936 -37.78035451832)
20169,3-57-mjp-6.4.R,17948,LINESTRING (144.877988852369 -37.7688291815855...,POINT (144.914787348936 -37.78035451832)
20170,3-57-mjp-3.4.R,17948,LINESTRING (144.877988852369 -37.7688291815855...,POINT (144.914787348936 -37.78035451832)


In [128]:
dfrt = pd.merge(df[bid]['routes'], df[bid]['trips'], on='route_id')
assert dfrt.apply(lambda x: str(x['route_short_name']) in x['route_id'], axis=1).all()
assert dfrt.apply(lambda x: x['route_id'] in x['shape_id'], axis=1).all()
assert dfrt.apply(lambda x: x['shape_id'] in x['trip_id'], axis=1).all()
dfrt['trip_id_x'] = dfrt['trip_id'].apply(lambda x: x.split('.'))
trip_id_x = dfrt['trip_id_x'].apply(len).unique()
assert len(trip_id_x) == 1
trip_id_count = trip_id_x[0]
for i in range(trip_id_count):
    dfrt[f'trip_id_{i}'] = dfrt['trip_id_x'].apply(lambda x: x[i])


In [100]:
df['3']['routes'].head(2)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,3-109-mjp-3,NaN,109,Port Melbourne - Box Hill,0,E87722,000000
1,3-109-mjp-4,NaN,109,Port Melbourne - Box Hill,0,E87722,000000


In [135]:
dfrt[['direction_id', 'trip_id_4']].value_counts()

direction_id  trip_id_4
0             H            25576
1             R            24795
Name: count, dtype: int64

In [150]:
df['3']['trips']['direction_id'].unique()

array([0, 1], dtype=int64)

In [142]:
df['5']['trips']['direction_id']

0       0
1       1
2       0
3       1
4       0
       ..
6726    0
6727    0
6728    0
6729    0
6730    1
Name: direction_id, Length: 6731, dtype: int64

In [126]:
assert dfrt.apply(lambda x: x['shape_id'] in x['trip_id'], axis=1).all()

In [125]:
dfrt['shape_id_1'].unique()

array(['3', '5', '8', '9', '10', '4', '1', '2', '6', '7', '11', '12',
       '13', '14'], dtype=object)

In [113]:
assert dfrt['shape_id_x'].apply(len).nunique()

1

In [106]:
dfrt['trip_id_x'].apply(len).unique()

array([5], dtype=int64)

In [104]:
dfrt['shape_id_2'] = dfrt['shape_id_1'].apply(lambda x: x[1])
dfrt['trip_id_2'] = dfrt['trip_id_1'].apply(lambda x: x[1])


0        [3-109-mjp-3, 3, H]
1        [3-109-mjp-3, 5, H]
2        [3-109-mjp-3, 5, H]
3        [3-109-mjp-3, 5, H]
4        [3-109-mjp-3, 5, H]
                ...         
50366     [3-96-mjp-7, 6, H]
50367    [3-96-mjp-7, 11, R]
50368     [3-96-mjp-7, 6, H]
50369     [3-96-mjp-7, 6, H]
50370    [3-96-mjp-7, 11, R]
Name: shape_id_1, Length: 50371, dtype: object

In [85]:
pd.merge(df['3']['routes'], df['3']['trips'], on='route_id').groupby(['route_short_name', 'trip_headsign', 'direction_id'])['shape_id'].unique()

route_short_name  trip_headsign                                       direction_id
1                 East Coburg to South Melbourne Beach                1               [3-1-mjp-3.9.R, 3-1-mjp-3.10.R, 3-1-mjp-3.11.R...
                  South Melbourne Beach to East Coburg                0               [3-1-mjp-3.3.H, 3-1-mjp-3.4.H, 3-1-mjp-3.5.H, ...
3                 East Malvern to Melbourne University                1               [3-3-mjp-3.5.R, 3-3-mjp-3.6.R, 3-3-mjp-4.5.R, ...
                  Melbourne University to East Malvern                0               [3-3-mjp-3.2.H, 3-3-mjp-3.3.H, 3-3-mjp-4.2.H, ...
5                 Malvern (Burke Road) to Melbourne University        1               [3-5-mjp-3.4.R, 3-5-mjp-4.4.R, 3-5-mjp-5.2.R, ...
                  Melbourne University to Malvern (Burke Road)        0               [3-5-mjp-3.2.H, 3-5-mjp-4.2.H, 3-5-mjp-5.1.H, ...
6                 Glen Iris to Moreland                               1               [3-6-mjp-3.7.R,